In [1]:
import cv2
import pandas as pd
from ultralytics import YOLO
from utils import Tracker

model = YOLO('yolov9c.pt')

In [2]:
class_list = ['person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed', 'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush']

In [3]:
tracker=Tracker()
count=0

In [4]:
video = cv2.VideoCapture('Videos/004_26031024.mp4')

fps = int(video.get(cv2.CAP_PROP_FPS))
width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Define the codec and create VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Codec for MP4
out = cv2.VideoWriter('Videos/video_with_boxes.mp4', fourcc, fps, (width, height))  

In [ ]:
#dictionary
down = {}
count_down=set()
while True:
    ret,frame = video.read()
    if not ret:
        break
    count += 1

    results=model.predict(frame)

    a=results[0].boxes.data
    a = a.detach().cpu().numpy()  
    px=pd.DataFrame(a).astype("float")
    
    list=[]
             
    #check if cars are detected and add their coordinates to the list
    for index,row in px.iterrows():
        d=int(row[5])
        c=class_list[d]
        if 'car' in c:
            x1=int(row[0])
            y1=int(row[1])
            x2=int(row[2])
            y2=int(row[3])
            list.append([x1,y1,x2,y2])
            

    
    bbox_id=tracker.update(list)
    #Print the boxes detected of cars in the current frame
    for bbox in bbox_id:
        x3,y3,x4,y4,id=bbox
        cx=int(x3+x4)//2
        cy=int(y3+y4)//2
        cv2.circle(frame,(cx,cy),4,(0,0,255),-1) #draw center points of bounding box
        cv2.rectangle(frame, (x3, y3), (x4, y4), (0, 255, 0), 2)  # Draw bounding box
        cv2.putText(frame,str(id),(cx,cy),cv2.FONT_HERSHEY_COMPLEX,0.8,(0,255,255),2)
        
        #y coordinate for red line
        y = 190
        x_left = 277
        x_right = 341
        offset = 10

        #defines when a card touched the red line
        if y < (cy + offset) and y > (cy - offset) and x_right < cx and x_left > cx:  
          #down[id]=cy   #cy is current position. saving the ids of the cars which are touching the red line first. 
          #This will tell us the traveling direction of the car.
          #if id in down:         
            #cv2.circle(frame,(cx,cy),4,(0,0,255),-1)
            cv2.putText(frame,"Touched",(cx,cy),cv2.FONT_HERSHEY_COMPLEX,0.8,(0,255,255),2)
            count_down.add(id) 

    # # line
    text_color = (255,255,255)  # white color for text
    red_color = (0, 0, 255)  # (B, G, R)   
    
    # line1
    cv2.line(frame,(277,190),(341,190),red_color,3)  #  starting cordinates and end of line cordinates
    cv2.putText(frame,('lane 3'),(277,180),cv2.FONT_HERSHEY_SIMPLEX, 0.5, text_color, 1, cv2.LINE_AA)    
    
    # line2
    cv2.line(frame,(230,222),(312,222),red_color,3)  #  starting cordinates and end of line cordinates
    cv2.putText(frame,('lane 2'),(230,212),cv2.FONT_HERSHEY_SIMPLEX, 0.5, text_color, 1, cv2.LINE_AA)    

    downwards = (len(count_down))
    cv2.putText(frame,('going down - ')+ str(downwards),(60,40),cv2.FONT_HERSHEY_SIMPLEX, 0.5, red_color, 1, cv2.LINE_AA) 
    
    
    cv2.imshow("frames", frame)
    if cv2.waitKey(1)&0xFF==27:
        break
video.release()
cv2.destroyAllWindows()


0: 544x640 1 car, 1 truck, 1 stop sign, 1452.0ms
Speed: 4.0ms preprocess, 1452.0ms inference, 4.0ms postprocess per image at shape (1, 3, 544, 640)

0: 544x640 1 car, 1 truck, 1 stop sign, 1421.0ms
Speed: 4.0ms preprocess, 1421.0ms inference, 2.0ms postprocess per image at shape (1, 3, 544, 640)

0: 544x640 1 car, 1 stop sign, 1406.2ms
Speed: 3.0ms preprocess, 1406.2ms inference, 2.0ms postprocess per image at shape (1, 3, 544, 640)

0: 544x640 1 car, 1 stop sign, 1409.0ms
Speed: 3.0ms preprocess, 1409.0ms inference, 3.0ms postprocess per image at shape (1, 3, 544, 640)

0: 544x640 2 cars, 1 stop sign, 1477.0ms
Speed: 8.0ms preprocess, 1477.0ms inference, 3.0ms postprocess per image at shape (1, 3, 544, 640)

0: 544x640 1 car, 1 stop sign, 1427.0ms
Speed: 4.8ms preprocess, 1427.0ms inference, 3.0ms postprocess per image at shape (1, 3, 544, 640)

0: 544x640 1 car, 1 stop sign, 1432.0ms
Speed: 3.0ms preprocess, 1432.0ms inference, 3.0ms postprocess per image at shape (1, 3, 544, 640)



In [23]:
model = YOLO('yolov10l.pt')

100%|██████████| 50.0M/50.0M [00:01<00:00, 32.1MB/s]


In [ ]:
class_list

In [ ]:
#dictionary
down = {}
count_down=set()
while True:
    ret,frame = video.read()
    if not ret:
        break
    count += 1

    results=model.predict(frame)

    a=results[0].boxes.data
    a = a.detach().cpu().numpy()  
    px=pd.DataFrame(a).astype("float")
    #print(px)
    
    list=[]
             
    for index,row in px.iterrows():
#        print(row) 
        x1=int(row[0])
        y1=int(row[1])
        x2=int(row[2])
        y2=int(row[3])
        d=int(row[5])
        c=class_list[d]
        if 'car' in c:
            list.append([x1,y1,x2,y2])
            


    bbox_id=tracker.update(list)
    #print(bbox_id)
    for bbox in bbox_id:
        x3,y3,x4,y4,id=bbox
        cx=int(x3+x4)//2
        cy=int(y3+y4)//2
        cv2.circle(frame,(cx,cy),4,(0,0,255),-1) #draw center points of bounding box
        cv2.rectangle(frame, (x3, y3), (x4, y4), (0, 255, 0), 2)  # Draw bounding box
        cv2.putText(frame,str(id),(cx,cy),cv2.FONT_HERSHEY_COMPLEX,0.8,(0,255,255),2)
        


        #y coordinate for red line
        y = 190
        x_left = 277
        x_right = 341
        #
        offset = 7
        ''' condition for red line '''
        if y < (cy + offset) and y > (cy - offset) and x_right < cx and x_left > cx:
          ''' this if condition is putting the id and the circle on the object when the center of the object touched the red line.'''
          
          down[id]=cy   #cy is current position. saving the ids of the cars which are touching the red line first. 
          #This will tell us the travelling direction of the car.
          if id in down:         
           cv2.circle(frame,(cx,cy),4,(0,0,255),-1)
           cv2.putText(frame,str(id),(cx,cy),cv2.FONT_HERSHEY_COMPLEX,0.8,(0,255,255),2)
           count_down.add(id) 

    # # line
    text_color = (255,255,255)  # white color for text
    red_color = (0, 0, 255)  # (B, G, R)   
    
    # line1
    cv2.line(frame,(277,190),(341,190),red_color,3)  #  starting cordinates and end of line cordinates
    cv2.putText(frame,('lane 3'),(277,180),cv2.FONT_HERSHEY_SIMPLEX, 0.5, text_color, 1, cv2.LINE_AA)    
    
    # line2
    cv2.line(frame,(230,222),(312,222),red_color,3)  #  starting cordinates and end of line cordinates
    cv2.putText(frame,('lane 2'),(230,212),cv2.FONT_HERSHEY_SIMPLEX, 0.5, text_color, 1, cv2.LINE_AA)    


    downwards = (len(count_down))
    cv2.putText(frame,('going down - ')+ str(downwards),(60,40),cv2.FONT_HERSHEY_SIMPLEX, 0.5, red_color, 1, cv2.LINE_AA) 
    
    
    cv2.imshow("frames", frame)
    if cv2.waitKey(1)&0xFF==27:
        break
video.release()
cv2.destroyAllWindows()


0: 544x640 (no detections), 1661.0ms
Speed: 3.0ms preprocess, 1661.0ms inference, 1.0ms postprocess per image at shape (1, 3, 544, 640)

0: 544x640 (no detections), 1658.0ms
Speed: 3.0ms preprocess, 1658.0ms inference, 0.0ms postprocess per image at shape (1, 3, 544, 640)

0: 544x640 (no detections), 1684.0ms
Speed: 3.0ms preprocess, 1684.0ms inference, 0.0ms postprocess per image at shape (1, 3, 544, 640)

0: 544x640 (no detections), 1652.0ms
Speed: 5.0ms preprocess, 1652.0ms inference, 0.0ms postprocess per image at shape (1, 3, 544, 640)

0: 544x640 (no detections), 1640.3ms
Speed: 3.0ms preprocess, 1640.3ms inference, 0.0ms postprocess per image at shape (1, 3, 544, 640)

0: 544x640 (no detections), 1657.0ms
Speed: 3.0ms preprocess, 1657.0ms inference, 0.0ms postprocess per image at shape (1, 3, 544, 640)

0: 544x640 (no detections), 1622.0ms
Speed: 3.0ms preprocess, 1622.0ms inference, 0.0ms postprocess per image at shape (1, 3, 544, 640)

0: 544x640 (no detections), 1619.0ms
Spe